In [1514]:
import pandas as pd
import sqlalchemy as sa
import requests
import json
import sqlalchemy
import nest_asyncio
from requests_html import AsyncHTMLSession

## CURRENT WEATHER API

In [1515]:
appid = '0ee1aeec809c143ac777fe7340414f60'
lat=43.6548
lon= -79.3883
units= 'metric'
limit = 10000
offset = 0

current_weather_api_url = 'https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&units={}&appid={}&limit={}&offset={}'.format(lat, lon, units, appid, limit, offset)

print(current_weather_api_url)

https://api.openweathermap.org/data/2.5/weather?lat=43.6548&lon=-79.3883&units=metric&appid=0ee1aeec809c143ac777fe7340414f60&limit=10000&offset=0


In [1516]:
api_response = requests.get(current_weather_api_url)

api_response

<Response [200]>

In [1517]:
data = api_response.json()
data

{'coord': {'lon': -79.3883, 'lat': 43.6548},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 27.59,
  'feels_like': 29.36,
  'temp_min': 24.87,
  'temp_max': 30.15,
  'pressure': 1005,
  'humidity': 65},
 'visibility': 10000,
 'wind': {'speed': 8.75, 'deg': 240, 'gust': 11.83},
 'clouds': {'all': 75},
 'dt': 1691866342,
 'sys': {'type': 1,
  'id': 718,
  'country': 'CA',
  'sunrise': 1691835513,
  'sunset': 1691886413},
 'timezone': -14400,
 'id': 6167863,
 'name': 'Downtown Toronto',
 'cod': 200}

In [1518]:
from datetime import datetime

In [1519]:
data_list = []

dt = pd.to_datetime(data['dt'], unit='s')
sys_sunrise = pd.to_datetime(data['sys']['sunrise'], unit='s')
sys_sunset = pd.to_datetime(data['sys']['sunset'], unit='s')

data_dict = {
    'timezone': data['timezone'],
    'id': data['id'],
    'name': data['name'],
    'coord_lon': data['coord']['lon'],
    'coord_lat': data['coord']['lat'],
    'cod': data['cod'],
    'weather_main': data['weather'][0]['main'],
    'weather_description': data['weather'][0]['description'],
    'weather_icon': data['weather'][0]['icon'],
    'base': data['base'],
    'main_temp': data['main']['temp'],
    'main_feels_like': data['main']['feels_like'],
    'main_temp_min': data['main']['temp_min'],
    'main_temp_max': data['main']['temp_max'],
    'main_pressure': data['main']['pressure'],
    'main_humidity': data['main']['humidity'],
    'visibility': data['visibility'],
    'wind_speed': data['wind']['speed'],
    'wind_deg': data['wind']['deg'],
    'clouds_all': data['clouds']['all'],
    'dt': dt,
    'sys_id': data['sys']['id'],
    'sys_country': data['sys']['country'],
    'sys_sunrise': sys_sunrise,
    'sys_sunset': sys_sunset
}

data_list.append(data_dict)

df = pd.DataFrame(data_list)

print(df)

   timezone       id              name  coord_lon  coord_lat  cod   
0    -14400  6167863  Downtown Toronto   -79.3883    43.6548  200  \

  weather_main weather_description weather_icon      base  ...  main_humidity   
0       Clouds       broken clouds          04d  stations  ...             65  \

   visibility  wind_speed  wind_deg  clouds_all                  dt  sys_id   
0       10000        8.75       240          75 2023-08-12 18:52:22     718  \

   sys_country         sys_sunrise          sys_sunset  
0           CA 2023-08-12 10:18:33 2023-08-13 00:26:53  

[1 rows x 25 columns]


In [1520]:
current_weather = pd.DataFrame(df)

print(current_weather)

   timezone       id              name  coord_lon  coord_lat  cod   
0    -14400  6167863  Downtown Toronto   -79.3883    43.6548  200  \

  weather_main weather_description weather_icon      base  ...  main_humidity   
0       Clouds       broken clouds          04d  stations  ...             65  \

   visibility  wind_speed  wind_deg  clouds_all                  dt  sys_id   
0       10000        8.75       240          75 2023-08-12 18:52:22     718  \

   sys_country         sys_sunrise          sys_sunset  
0           CA 2023-08-12 10:18:33 2023-08-13 00:26:53  

[1 rows x 25 columns]


In [1521]:
db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'      : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'      : '5432',
    'username'  : 'mredshaw',
    'password'  : '2023!Schulich',
    'database'  : 'mredshaw_db'
}

In [1522]:
db_connection_url = sa.engine.URL.create(
    drivername = db_secret['drivername'],
    username   = db_secret['username'],
    password   = db_secret['password'],
    host       = db_secret['host'],
    port       = db_secret['port'],
    database   = db_secret['database']
)

In [1523]:
engine = sa.create_engine(db_connection_url)

In [1524]:
#with engine.connect() as connection:
#    connection.execute('CREATE SCHEMA IF NOT EXISTS uploads')

In [1525]:
current_weather

,timezone,id,name,coord_lon,coord_lat,cod,weather_main,weather_description,weather_icon,base,...,main_humidity,visibility,wind_speed,wind_deg,clouds_all,dt,sys_id,sys_country,sys_sunrise,sys_sunset
0,-14400,6167863,Downtown Toronto,-79.3883,43.6548,200,Clouds,broken clouds,04d,stations,...,65,10000,8.75,240,75,2023-08-12 18:52:22,718,CA,2023-08-12 10:18:33,2023-08-13 00:26:53


In [1526]:
current_weather.dtypes

timezone                        int64
id                              int64
name                           object
coord_lon                     float64
coord_lat                     float64
cod                             int64
weather_main                   object
weather_description            object
weather_icon                   object
base                           object
main_temp                     float64
main_feels_like               float64
main_temp_min                 float64
main_temp_max                 float64
main_pressure                   int64
main_humidity                   int64
visibility                      int64
wind_speed                    float64
wind_deg                        int64
clouds_all                      int64
dt                     datetime64[ns]
sys_id                          int64
sys_country                    object
sys_sunrise            datetime64[ns]
sys_sunset             datetime64[ns]
dtype: object

In [1527]:
current_weather.to_sql(
    name = 'current_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
    'timezone': sa.types.INTEGER,
    'id': sa.types.INTEGER,
    'name': sa.types.VARCHAR(255),
    'coord_lon': sa.types.FLOAT,
    'coord_lat': sa.types.FLOAT,
    'cod': sa.types.INTEGER,
    'weather_main': sa.types.VARCHAR(255),
    'weather_description': sa.types.VARCHAR(255),
    'weather_icon': sa.types.VARCHAR(255),
    'base': sa.types.VARCHAR(255),
    'main_temp': sa.types.FLOAT,
    'main_feels_like': sa.types.FLOAT,
    'main_temp_min': sa.types.FLOAT,
    'main_temp_max': sa.types.FLOAT,
    'main_pressure': sa.types.INTEGER,
    'main_humidity': sa.types.INTEGER,
    'visibility': sa.types.INTEGER,
    'wind_speed': sa.types.FLOAT,
    'wind_deg': sa.types.INTEGER,
    'clouds_all': sa.types.INTEGER,
    'dt': sa.types.TIMESTAMP,
    'sys_id': sa.types.INTEGER,
    'sys_country': sa.types.VARCHAR(255),
    'sys_sunrise': sa.types.TIMESTAMP,  # Change this to TIMESTAMP
    'sys_sunset': sa.types.TIMESTAMP   # Change this to TIMESTAMP

})

1

## FORECAST WEATHER API

In [1528]:
appid = '0ee1aeec809c143ac777fe7340414f60'
lat=43.6548
lon= -79.3883
limit = 10000
offset = 0

forecast_weather_api_url = 'https://api.openweathermap.org/data/2.5/forecast?lat={}&lon={}&appid={}&limit={}&offset={}'.format(lat, lon, appid, limit, offset)

print(forecast_weather_api_url)

https://api.openweathermap.org/data/2.5/forecast?lat=43.6548&lon=-79.3883&appid=0ee1aeec809c143ac777fe7340414f60&limit=10000&offset=0


In [1529]:
api_response_forecast = requests.get(forecast_weather_api_url)

api_response_forecast

<Response [200]>

In [1530]:
forecast_data = api_response_forecast.json()
forecast_data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1691874000,
   'main': {'temp': 300.79,
    'feels_like': 302.08,
    'temp_min': 300.79,
    'temp_max': 300.9,
    'pressure': 1005,
    'sea_level': 1005,
    'grnd_level': 995,
    'humidity': 60,
    'temp_kf': -0.11},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04d'}],
   'clouds': {'all': 61},
   'wind': {'speed': 5.77, 'deg': 256, 'gust': 8.7},
   'visibility': 10000,
   'pop': 0.46,
   'sys': {'pod': 'd'},
   'dt_txt': '2023-08-12 21:00:00'},
  {'dt': 1691884800,
   'main': {'temp': 298.52,
    'feels_like': 298.73,
    'temp_min': 297.41,
    'temp_max': 298.52,
    'pressure': 1006,
    'sea_level': 1006,
    'grnd_level': 995,
    'humidity': 62,
    'temp_kf': 1.11},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'clouds': {'all': 38},
   'wind': {'speed': 3.94, 'deg': 270, 'gust': 6.95},
   'visibil

In [1531]:
data_list = []

for item in forecast_data['list']:
    dt = pd.to_datetime(item['dt'], unit='s')
    main_temp = item['main']['temp']
    main_feels_like = item['main']['feels_like']
    temp_min = item['main']['temp_min']
    temp_max = item['main']['temp_max']
    pressure = item['main']['pressure']
    humidity = item['main']['humidity']
    
    weather_id = item['weather'][0]['id']
    weather_main = item['weather'][0]['main']
    weather_description = item['weather'][0]['description']
    weather_icon = item['weather'][0]['icon']
    
    data_list.append({
        'dt': dt,
        'main_temp': main_temp,
        'main_feels_like': main_feels_like,
        'temp_min': temp_min,
        'temp_max': temp_max,
        'pressure': pressure,
        'humidity': humidity,
        'weather_id': weather_id,
        'weather_main': weather_main,
        'weather_description': weather_description,
        'weather_icon': weather_icon
    })

forecast_df = pd.DataFrame(data_list)

print(forecast_df)


                    dt  main_temp  main_feels_like  temp_min  temp_max   
0  2023-08-12 21:00:00     300.79           302.08    300.79    300.90  \
1  2023-08-13 00:00:00     298.52           298.73    297.41    298.52   
2  2023-08-13 03:00:00     294.42           294.56    294.42    294.42   
3  2023-08-13 06:00:00     292.34           292.17    292.34    292.34   
4  2023-08-13 09:00:00     290.91           290.75    290.91    290.91   
5  2023-08-13 12:00:00     291.50           291.40    291.50    291.50   
6  2023-08-13 15:00:00     295.08           294.79    295.08    295.08   
7  2023-08-13 18:00:00     298.14           297.90    298.14    298.14   
8  2023-08-13 21:00:00     298.69           298.50    298.69    298.69   
9  2023-08-14 00:00:00     295.50           295.46    295.50    295.50   
10 2023-08-14 03:00:00     292.52           292.52    292.52    292.52   
11 2023-08-14 06:00:00     291.02           290.95    291.02    291.02   
12 2023-08-14 09:00:00     290.15     

In [1532]:
forecast_df.shape

(40, 11)

In [1533]:
forecast_weather = pd.DataFrame(forecast_df)

print(forecast_weather)

                    dt  main_temp  main_feels_like  temp_min  temp_max   
0  2023-08-12 21:00:00     300.79           302.08    300.79    300.90  \
1  2023-08-13 00:00:00     298.52           298.73    297.41    298.52   
2  2023-08-13 03:00:00     294.42           294.56    294.42    294.42   
3  2023-08-13 06:00:00     292.34           292.17    292.34    292.34   
4  2023-08-13 09:00:00     290.91           290.75    290.91    290.91   
5  2023-08-13 12:00:00     291.50           291.40    291.50    291.50   
6  2023-08-13 15:00:00     295.08           294.79    295.08    295.08   
7  2023-08-13 18:00:00     298.14           297.90    298.14    298.14   
8  2023-08-13 21:00:00     298.69           298.50    298.69    298.69   
9  2023-08-14 00:00:00     295.50           295.46    295.50    295.50   
10 2023-08-14 03:00:00     292.52           292.52    292.52    292.52   
11 2023-08-14 06:00:00     291.02           290.95    291.02    291.02   
12 2023-08-14 09:00:00     290.15     

In [1534]:
forecast_weather.dtypes

dt                     datetime64[ns]
main_temp                     float64
main_feels_like               float64
temp_min                      float64
temp_max                      float64
pressure                        int64
humidity                        int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [1535]:
forecast_weather.to_sql(
    name = 'forecast_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
          'dt' : sa.types.TIMESTAMP,
          'main_temp' : sa.types.FLOAT,
          'main_feels_like': sa.types.FLOAT,
          'temp_min': sa.types.FLOAT,
          'temp_max': sa.types.FLOAT,
          'pressure': sa.types.INTEGER,
          'humidity': sa.types.INTEGER,
          'weather_id': sa.types.INTEGER,
          'weather_main': sa.types.VARCHAR(255),
          'weather_description': sa.types.VARCHAR(255),
          'weather_icon': sa.types.VARCHAR(255)
})

40

## HISTORICAL WEATHER API

In [1536]:
key = '6fe878d1ff8d425185816caa82e1e9b8'
latitude= 43.6548
longitude= -79.3883
start_date = '2023-07-24'
end_date = '2023-08-07'
hourly = 'temperature_2m'
limit = 10000
offset = 0

history_weather_api_url2 = 'https://archive-api.open-meteo.com/v1/archive?latitude=43.6548&longitude=-79.3883&start_date=2023-07-24&end_date=2023-08-07&hourly=temperature_2m&daily=temperature_2m_max,temperature_2m_min,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,windspeed_10m_max,windgusts_10m_max&timezone=America%2FNew_York'

print(history_weather_api_url2)

https://archive-api.open-meteo.com/v1/archive?latitude=43.6548&longitude=-79.3883&start_date=2023-07-24&end_date=2023-08-07&hourly=temperature_2m&daily=temperature_2m_max,temperature_2m_min,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,windspeed_10m_max,windgusts_10m_max&timezone=America%2FNew_York


In [1537]:
history_api_response2 = requests.get(history_weather_api_url2)

history_api_response2

<Response [200]>

In [1538]:
history_data2 = history_api_response2.json()
history_data2

{'latitude': 43.699997,
 'longitude': -79.4,
 'generationtime_ms': 0.6469488143920898,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 95.0,
 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'},
 'hourly': {'time': ['2023-07-24T00:00',
   '2023-07-24T01:00',
   '2023-07-24T02:00',
   '2023-07-24T03:00',
   '2023-07-24T04:00',
   '2023-07-24T05:00',
   '2023-07-24T06:00',
   '2023-07-24T07:00',
   '2023-07-24T08:00',
   '2023-07-24T09:00',
   '2023-07-24T10:00',
   '2023-07-24T11:00',
   '2023-07-24T12:00',
   '2023-07-24T13:00',
   '2023-07-24T14:00',
   '2023-07-24T15:00',
   '2023-07-24T16:00',
   '2023-07-24T17:00',
   '2023-07-24T18:00',
   '2023-07-24T19:00',
   '2023-07-24T20:00',
   '2023-07-24T21:00',
   '2023-07-24T22:00',
   '2023-07-24T23:00',
   '2023-07-25T00:00',
   '2023-07-25T01:00',
   '2023-07-25T02:00',
   '2023-07-25T03:00',
   '2023-07-25T04:00',
   '2023-07-25T05:00',
   '2023-07-25T06:00',
   

In [1539]:
hist_df = pd.DataFrame(history_data2)

In [1540]:
hist_df['daily_units'].unique()

array(['iso8601', nan, '°C', 'mm', 'cm', 'km/h'], dtype=object)

In [1541]:
hist_df.columns

Index(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds',
       'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units',
       'hourly', 'daily_units', 'daily'],
      dtype='object')

In [1542]:
history_data2['daily_units']

{'time': 'iso8601',
 'temperature_2m_max': '°C',
 'temperature_2m_min': '°C',
 'sunrise': 'iso8601',
 'sunset': 'iso8601',
 'precipitation_sum': 'mm',
 'rain_sum': 'mm',
 'snowfall_sum': 'cm',
 'windspeed_10m_max': 'km/h',
 'windgusts_10m_max': 'km/h'}

In [1554]:
data_list = []

latitude = history_data2['latitude']
longitude = history_data2['longitude']
generationtime_ms = history_data2['generationtime_ms']
utc_offset_seconds = history_data2['utc_offset_seconds']
timezone = history_data2['timezone']
timezone_abbreviation = history_data2['timezone_abbreviation']
elevation = history_data2['elevation']
hourly_units_time = history_data2['hourly_units']['time']
hourly_units_temp = history_data2['hourly_units']['temperature_2m']
hourly = history_data2['hourly']['time']
daily_units_time = history_data2['daily_units']['time']
temperature_2m_max_unit = history_data2['daily_units']['temperature_2m_max']
temperature_2m_min_unit = history_data2['daily_units']['temperature_2m_min']
sunrise_unit = history_data2['daily_units']['sunrise']
sunset_unit = history_data2['daily_units']['sunset']
precipitation_sum_unit = history_data2['daily_units']['precipitation_sum']
rain_sum_unit = history_data2['daily_units']['rain_sum']
snowfall_sum_unit = history_data2['daily_units']['snowfall_sum']
windspeed_10m_max_unit = history_data2['daily_units']['windspeed_10m_max']
windgusts_10m_max_unit = history_data2['daily_units']['windgusts_10m_max']


data_list.append({
    'latitude': latitude,
    'longitude': longitude,
    'generationtime_ms': generationtime_ms,
    'utc_offset_seconds': utc_offset_seconds,
    'timezone': timezone,
    'timezone_abbreviation': timezone_abbreviation,
    'elevation': elevation,
    'hourly_time': timestamp,
    'hourly_temp': hourly_units_temp[idx] if idx < len(hourly_units_temp) else None,
    'hourly': hourly,
    'daily_units_time': daily_units_time,
    'temperature_2m_max_unit': temperature_2m_max_unit,
    'temperature_2m_min_unit': temperature_2m_min_unit,
    'sunrise_unit': sunrise_unit,
    'sunset_unit': sunset_unit,
    'precipitation_sum_unit': precipitation_sum_unit,
    'rain_sum_unit': rain_sum_unit,
    'snowfall_sum_unit': snowfall_sum_unit,
    'windspeed_10m_max_unit': windspeed_10m_max_unit,
    'windgusts_10m_max_unit': windgusts_10m_max_unit
})

history_data2_df = pd.DataFrame(data_list)

print(history_data2_df)

    latitude  longitude  generationtime_ms  utc_offset_seconds   
0  43.699997      -79.4           0.646949              -14400  \

           timezone timezone_abbreviation  elevation       hourly_time   
0  America/New_York                   EDT       95.0  2023-08-07T23:00  \

  hourly_temp                                             hourly   
0        None  [2023-07-24T00:00, 2023-07-24T01:00, 2023-07-2...  \

  daily_units_time temperature_2m_max_unit temperature_2m_min_unit   
0          iso8601                      °C                      °C  \

  sunrise_unit sunset_unit precipitation_sum_unit rain_sum_unit   
0      iso8601     iso8601                     mm            mm  \

  snowfall_sum_unit windspeed_10m_max_unit windgusts_10m_max_unit  
0                cm                   km/h                   km/h  


In [1555]:
history_data2_df

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,hourly_time,hourly_temp,hourly,daily_units_time,temperature_2m_max_unit,temperature_2m_min_unit,sunrise_unit,sunset_unit,precipitation_sum_unit,rain_sum_unit,snowfall_sum_unit,windspeed_10m_max_unit,windgusts_10m_max_unit
0,43.699997,-79.4,0.646949,-14400,America/New_York,EDT,95.0,2023-08-07T23:00,None,"[2023-07-24T00:00, 2023-07-24T01:00, 2023-07-2...",iso8601,°C,°C,iso8601,iso8601,mm,mm,cm,km/h,km/h


In [1545]:
history_data2_df.dtypes

latitude                   float64
longitude                  float64
generationtime_ms          float64
utc_offset_seconds           int64
timezone                    object
timezone_abbreviation       object
elevation                  float64
hourly_units_time           object
hourly_units_temp           object
hourly                      object
daily_units_time            object
temperature_2m_max_unit     object
temperature_2m_min_unit     object
sunrise_unit                object
sunset_unit                 object
precipitation_sum_unit      object
rain_sum_unit               object
snowfall_sum_unit           object
windspeed_10m_max_unit      object
windgusts_10m_max_unit      object
dtype: object

In [1546]:
history_data2_df.to_sql(
    name='history_weather', 
    schema='uploads',
    con=engine,
    if_exists='replace',
    index=False,
    method='multi',
    dtype={
        'latitude': sa.types.FLOAT,
        'longitude': sa.types.FLOAT,
        'generationtime_ms': sa.types.FLOAT,
        'utc_offset_seconds': sa.types.INTEGER,
        'timezone': sa.types.VARCHAR(255),
        'timezone_abbreviation': sa.types.VARCHAR(255),
        'elevation': sa.types.FLOAT,
        'hourly_units_time': sa.types.VARCHAR(255),
        'hourly_units_temp': sa.types.VARCHAR(255),
        'hourly': sa.types.JSON,
        'daily_units_time': sa.types.VARCHAR(255),
        'temperature_2m_max_unit': sa.types.VARCHAR(255),
        'temperature_2m_min_unit': sa.types.VARCHAR(255),
        'sunrise_unit': sa.types.VARCHAR(255),
        'sunset_unit': sa.types.VARCHAR(255),
        'precipitation_sum_unit': sa.types.VARCHAR(255),
        'rain_sum_unit': sa.types.VARCHAR(255),
        'snowfall_sum_unit': sa.types.VARCHAR(255),
        'windspeed_10m_max_unit': sa.types.VARCHAR(255),
        'windgusts_10m_max_unit': sa.types.VARCHAR(255)
    }
)

1

In [1547]:
history_data2.to_sql(
    name = 'history_weather', 
    schema = 'uploads',
    con = engine,
    if_exists = 'replace',
    index = False,
    method = 'multi',
      dtype = {
          'dt' : sa.types.TIMESTAMP,
          'main_temp' : sa.types.FLOAT,
          'main_feels_like': sa.types.FLOAT,
          'temp_min': sa.types.FLOAT,
          'temp_max': sa.types.FLOAT,
          'pressure': sa.types.INTEGER,
          'humidity': sa.types.INTEGER,
          'weather_id': sa.types.INTEGER,
          'weather_main': sa.types.VARCHAR(255),
          'weather_description': sa.types.VARCHAR(255),
          'weather_icon': sa.types.VARCHAR(255)
})

AttributeError: 'dict' object has no attribute 'to_sql'

In [ ]:
key = '617ed86c2036463688f42f2f808227bb'
lat=43.6548
lon= -79.3883
start_date = '2023-01-01:00'
end_date = '2023-08-09:00'
limit = 10000
offset = 0

history_weather_api_url = 'https://api.weatherbit.io/v2.0/history/daily?lat={}&lon={}&key={}&start_date={}&end_date={}&limit={}&offset={}'.format(lat, lon, key, start_date, end_date,limit, offset)

print(history_weather_api_url)

https://api.weatherbit.io/v2.0/history/daily?lat=43.6548&lon=-79.3883&key=617ed86c2036463688f42f2f808227bb&start_date=2023-01-01:00&end_date=2023-08-09:00&limit=10000&offset=0


In [ ]:
#history_api_response = requests.get(history_weather_api_url)

#history_api_response

<Response [429]>

In [ ]:
#history_data = history_api_response.json()
#history_data

{'status_code': 429,
 'status_message': 'Your request count (1984) is over the allowed limit of 1500 per day - Upgrade your key, or retry after 312.7 minutes'}

In [ ]:
#history_data['data']

KeyError: 'data'

In [ ]:
#history_weather = pd.DataFrame(history_data['data'])
#history_weather

KeyError: 'data'

In [ ]:
#history_weather.dtypes

clouds               int64
datetime            object
dewpt              float64
dhi                  int64
dni                  int64
ghi                  int64
max_dhi              int64
max_dni              int64
max_ghi              int64
max_temp           float64
max_temp_ts          int64
max_uv             float64
max_wind_dir         int64
max_wind_spd       float64
max_wind_spd_ts      int64
min_temp           float64
min_temp_ts          int64
precip             float64
precip_gpm         float64
pres                 int64
revision_status     object
rh                 float64
slp                  int64
snow               float64
snow_depth         float64
solar_rad            int64
t_dhi                int64
t_dni                int64
t_ghi                int64
t_solar_rad          int64
temp               float64
ts                   int64
wind_dir             int64
wind_gust_spd      float64
wind_spd           float64
dtype: object

In [ ]:
#history_weather['datetime'] = pd.to_datetime(history_weather['datetime'])

In [ ]:
#history_weather.to_sql(
#    name='history_weather',
#    schema='uploads',
#    con=engine,
#    if_exists='replace',
#    index=False,
#    method='multi',
#    dtype={
#        'clouds': sa.types.INTEGER,
#        'datetime': sa.types.TIMESTAMP,
#        'dewpt': sa.types.FLOAT,
#        'dhi': sa.types.INTEGER,
#        'dni': sa.types.INTEGER,
#        'ghi': sa.types.INTEGER,
#        'max_dhi': sa.types.INTEGER,
#        'max_dni': sa.types.INTEGER,
#        'max_ghi': sa.types.INTEGER,
#        'max_temp': sa.types.FLOAT,
#        'max_temp_ts': sa.types.INTEGER,
#        'max_uv': sa.types.FLOAT,
#        'max_wind_dir': sa.types.INTEGER,
#        'max_wind_spd': sa.types.FLOAT,
#        'max_wind_spd_ts': sa.types.INTEGER,
#        'min_temp': sa.types.FLOAT,
#        'min_temp_ts': sa.types.INTEGER,
#        'precip': sa.types.FLOAT,
#        'precip_gpm': sa.types.FLOAT,
#        'pres': sa.types.INTEGER,
#        'revision_status': sa.types.VARCHAR(255),
#        'rh': sa.types.FLOAT,
#        'slp': sa.types.INTEGER,
#        'snow': sa.types.FLOAT,
#        'snow_depth': sa.types.FLOAT,
#        'solar_rad': sa.types.INTEGER,
#        't_dhi': sa.types.INTEGER,
#        't_dni': sa.types.INTEGER,
#        't_ghi': sa.types.INTEGER,
#        't_solar_rad': sa.types.INTEGER,
#        'temp': sa.types.FLOAT,
#        'ts': sa.types.INTEGER,
#        'wind_dir': sa.types.INTEGER,
#        'wind_gust_spd': sa.types.FLOAT,
#        'wind_spd': sa.types.FLOAT
#    }
#)

220

## WEATHER ALERT FETCH

In [ ]:
nest_asyncio.apply()
asession = AsyncHTMLSession()

In [ ]:
async def fetch_website(weburl):
    url = weburl
    r = await asession.get(url)
    
    if r.status_code == 200:
        print(f"Successfully fetched data from {url}")
    else:
        print(f"Failed to fetch data from {url}, status code: {r.status_code}")

    await r.html.arender(sleep=1)
    
    href_list = r.html.find('pre a[href]')
    href_values = [link.attrs['href'] for link in href_list]

    return href_values

In [ ]:
weburl = 'https://dd.weather.gc.ca/alerts/cap/'
href_values_list = asession.loop.run_until_complete(fetch_website(weburl))
latest_date = href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/


[INFO] Starting Chromium download.
100%|██████████| 86.8M/86.8M [00:14<00:00, 6.12Mb/s]
[INFO] Beginning extraction
[INFO] Chromium extracted to: /Users/mikeredshaw/Library/Application Support/pyppeteer/local-chromium/588429


In [ ]:
print("----------------------")
print("Today's Date : ",latest_date[0:4],"-",latest_date[4:6],"-",latest_date[6:8])
print("----------------------")
new_url = weburl+latest_date+'CWTO/'
print(new_url)

----------------------
Today's Date :  2023 - 08 - 12
----------------------
https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/


In [ ]:
new_href_values_list = asession.loop.run_until_complete(fetch_website(new_url))
latest_file = new_href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/


In [ ]:
latest_file = new_href_values_list[-1]
print(latest_file)

04/


In [ ]:
newz_url = new_url+latest_file
print(newz_url)

https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/04/


In [ ]:
new_href_values_list = asession.loop.run_until_complete(fetch_website(newz_url))
latest_datz = new_href_values_list[-1]

Successfully fetched data from https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/04/


In [ ]:
latest_datz = new_href_values_list[-1]
print(latest_datz)

T_WHCN13_C_CWTO_202308120415_2845420445.cap


In [ ]:
latest_file = newz_url+latest_datz

In [ ]:
print(latest_file)

https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/04/T_WHCN13_C_CWTO_202308120415_2845420445.cap


In [ ]:
req = requests.get(latest_file)

In [ ]:
req.url

'https://dd.weather.gc.ca/alerts/cap/20230812/CWTO/04/T_WHCN13_C_CWTO_202308120415_2845420445.cap'

In [ ]:
import os

directory_path = "/Users/mikeredshaw/Documents/Schulich MBAN/MMAI 5100 - Database Fundamentals/Weather API" # replace the file path
latest_datz = "example_file.xml" 
full_path = os.path.join(directory_path, latest_datz)

In [ ]:
with open(full_path, 'wb') as f:
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

In [ ]:
try:
    with open (full_path) as file:
        print("We can read the file John, Mike, Gabe, and Rahul")
except FileNotFoundError:
    print("I need help Boys!!")

We can read the file John, Mike, Gabe, and Rahul


In [ ]:
import xmltodict

with open(full_path, 'r') as xml_file:
    xml_data = xml_file.read()

In [ ]:
data_dict = xmltodict.parse(xml_data)

In [ ]:
import xmltodict

with open(full_path, 'r') as xml_file:
    xml_data = xml_file.read()

data_dict = xmltodict.parse(xml_data)

polygon_list = []


for info in data_dict['alert']['info']:
    if info.get('language') == 'en-CA':
        print("Language:", info['language'])
        print("Category:", info['category'])
        print("Event:", info['event'])
        print("Response Type:", info['responseType'])
        print("Urgency:", info['urgency'])
        print("Severity:", info['severity'])
        print("Certainty:", info['certainty'])
        print("Audience:", info['audience'])
        print("Effective:", info['effective'])
        print("Expires:", info['expires'])
        print("Sender Name:", info['senderName'])
        print("Headline:", info['headline'])
        print("Description:", info['description'])
        
        for area in info['area']:
            polygon = area['polygon']
            polygon_list.append(polygon)

Language: en-CA
Category: Met
Event: squall
Response Type: Monitor
Urgency: Expected
Severity: Severe
Certainty: Possible
Audience: general public
Effective: 2023-08-12T04:15:27-00:00
Expires: 2023-08-12T12:59:27-00:00
Sender Name: Environment Canada
Headline: squall watch in effect
Description: Conditions are favourable for the development of squalls with wind gusts up to 40 knots, frequent lightning and hail.

Scattered strong thunderstorms are expected tonight.

###

Large hail can damage vessels and cause injury.

Please continue to monitor alerts and forecasts issued by Environment Canada. For more information monitor Canadian Coast Guard radio or Weatheradio stations.


In [ ]:
print("Polygon List:", polygon_list)

Polygon List: ['42.0512,-83.1462 42.0537,-83.1109 42.0629,-83.0565 42.0167,-82.9069 42.0946,-82.6165 42.2319,-82.3391 42.5988,-81.7125 42.784,-81.2109 42.66,-81.1553 42.2591,-80.9327 42.2075,-81.2457 41.6765,-82.3974 41.6765,-82.6797 41.8635,-83.0691 42.0409,-83.1496 42.0512,-83.1462', '42.5527,-82.5993 42.5558,-82.5959 42.574,-82.5753 42.5798,-82.5688 42.5947,-82.5519 42.6139,-82.5225 42.6123,-82.5199 42.6113,-82.5182 42.6108,-82.5175 42.6095,-82.5154 42.5628,-82.5505 42.5505,-82.5386 42.5262,-82.515 42.5358,-82.4634 42.5188,-82.3954 42.3937,-82.3695 42.2795,-82.4137 42.2849,-82.5491 42.2679,-82.7164 42.3254,-82.9097 42.3377,-82.9189 42.3422,-82.9211 42.3485,-82.9249 42.3512,-82.9263 42.3586,-82.9303 42.3828,-82.8374 42.5447,-82.6736 42.5632,-82.6314 42.5592,-82.6191 42.5555,-82.6079 42.5527,-82.5993', '43.0052,-82.3823 43.0113,-82.4137 43.5911,-82.1227 44.7096,-82.3729 44.7041,-81.2898 44.7035,-81.273 44.7061,-81.2189 44.5146,-81.2158 44.3362,-81.4092 44.0654,-81.6693 43.8003,-81.641

In [ ]:
polygon_str = polygon_list[0]
points_str = polygon_str.split()

polygon_points = []

for point_str in points_str:
    lat, lon = map(float, point_str.split(','))
    polygon_points.append((lat, lon))

print(polygon_points)

[(42.0512, -83.1462), (42.0537, -83.1109), (42.0629, -83.0565), (42.0167, -82.9069), (42.0946, -82.6165), (42.2319, -82.3391), (42.5988, -81.7125), (42.784, -81.2109), (42.66, -81.1553), (42.2591, -80.9327), (42.2075, -81.2457), (41.6765, -82.3974), (41.6765, -82.6797), (41.8635, -83.0691), (42.0409, -83.1496), (42.0512, -83.1462)]


In [ ]:
def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False
    p1x, p1y = poly[0]
    for i in range(n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xints = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xints:
                        inside = 0
        p1x, p1y = p2x, p2y
    return 1


point_to_check = (43.6548, -79.3883)  

is_inside = point_inside_polygon(point_to_check[1], point_to_check[0], polygon_points)

if is_inside:
    print("The point is inside the polygon.")
else:
    print("The point is outside the polygon.")

The point is inside the polygon.
